In [39]:
import numpy as np
import pandas as pd

In [ ]:
dataset = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/2 Sentiment Analysis/a1_RestaurantReviews_HistoricDump.tsv",delimiter = '\t', quoting = 3)
dataset.head(10)

In [ ]:
dataset.shape

Data Cleaning

In [ ]:
import re
import nltk

nltk.download('stopwords')

from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

ps = PorterStemmer()

all_stopwords = stopwords.words('english')
all_stopwords.remove('not')

In [ ]:
corpus = []

for i in range(0,900):
  review = re.sub('[^a-zA-Z]',' ',dataset['Review'][i])
  review = review.lower()
  review = review.split()
  review = [ps.stem(word) for word in review if not word in set(all_stopwords)]
  review = ' '.join(review)
  corpus.append(review)

corpus

Data Transformation

In [44]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features = 1420)

In [45]:
x = cv.fit_transform(corpus).toarray()
y = dataset.iloc[:,-1].values

In [46]:
# saving Bow dictionary for later use in drive
import pickle
bow_path = "/content/drive/MyDrive/Colab Notebooks/2 Sentiment Analysis/Bow_sentiment model.pkl"
pickle.dump(cv, open(bow_path,"wb"))

In [47]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.20,random_state = 0)

In [ ]:
from sklearn.naive_bayes import GaussianNB
classifier = GaussianNB()
classifier.fit(x_train,y_train)

In [ ]:
import joblib
joblib.dump(classifier, "/content/drive/MyDrive/Colab Notebooks/2 Sentiment Analysis/classifier_sentiment model")

In [ ]:
y_pred = classifier.predict(x_test)

from sklearn.metrics import confusion_matrix,accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)

In [ ]:
accuracy = accuracy_score(y_test, y_pred)
accuracy

In [ ]:
new_dataset = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/2 Sentiment Analysis/a2_RestaurantReviews_FreshDump.tsv",delimiter='\t',quoting=3)
new_dataset.head()

In [ ]:
new_dataset.shape

In [ ]:
new_corpus = []

for i in range(0,100):
  review = re.sub('[^a-zA-Z]',' ',new_dataset['Review'][i])
  review = review.lower()
  review = review.split()
  review = [ps.stem(word) for word in review if not word in set(all_stopwords)]
  review = ' '.join(review)
  new_corpus.append(review)

new_corpus

In [56]:
cvfile = "/content/drive/MyDrive/Colab Notebooks/2 Sentiment Analysis/Bow_sentiment model.pkl"
cv = pickle.load(open(cvfile,'rb'))

In [ ]:
new_x = cv.transform(new_corpus).toarray()
new_x.shape

In [58]:
classifier = joblib.load("/content/drive/MyDrive/Colab Notebooks/2 Sentiment Analysis/classifier_sentiment model")

In [ ]:
y_pred = classifier.predict(new_x)
y_pred

In [ ]:
new_dataset["predicted label"] = y_pred.tolist()
new_dataset.head()

In [61]:
dataset.to_csv("/content/drive/MyDrive/Colab Notebooks/2 Sentiment Analysis/Predicted_Sentiments_Fresh_Dump.tsv", sep='\t', encoding='UTF-8', index=False)